# LDA(Latent Dirichlet Allocation): 잠재 디리클레 할당

https://wikidocs.net/40710 참고

In [1]:
import pymysql

DB_HOST = 'localhost'
DB_USER = 'pyuser'
DB_PASSWD = 'pyuser'
DB_NAME = 'pyuser'

conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                   db=DB_NAME, charset='utf8')
curs = conn.cursor()
sql = """
    select * 
    from fmkorea
--    where newsDate between '20190101' and '20191231';

"""
curs.execute(sql)
table_data = curs.fetchall() 
conn.close()

In [2]:
import pandas as pd

df = pd.DataFrame(table_data,columns=['no','url','Date','Title','subtitle','Contents'])
# df = pd.DataFrame(table_data,columns=['Contents'])

# df = pd.DataFrame(table_data)
# df.columns=['no','url','Date','Title','subtitle','Contents']

# 데이터 유무 확인. 분석건수 확인. 
df.head()

df.info()  # 전체 데이터 건수 확인

df.shape   # 전체 데이터 건수 확인

# df [ 'Contents'   ]  <= Series타입,   df[ ['Contents'] ]  <== dataframe타입
sentences = df[ ['Contents'] ]   

# print로 type(text) 의 결과는 Dataframe타입이어야 함
print(type(sentences))
# <class 'pandas.core.frame.DataFrame'> 이 아니면 오류임

<class 'pandas.core.frame.DataFrame'>


라이브러리 추가 및 클래스 선언

In [3]:
# konlpy 설치 명령어
# !pip install konlpy

In [4]:
from konlpy.tag import Okt
okt=Okt()

명사추출: nouns 명사추출. phrases 어절 추출

In [5]:
tokenized_doc = sentences.apply(lambda row: okt.nouns(row['Contents']), axis=1)


단어 길이가 1이하인 단어들은 제거 (길이가 짧은 단어 ex. 은/는/이/가/을/를 등 제거)


In [6]:
tokenized_doc = tokenized_doc.apply(lambda x: [word for word in x if len(word) > 1])


분석에서 제거할 불필요한 단어들 기재

In [7]:
from nltk.corpus import stopwords

stop = ['연합뉴스','무단']
tokenized_doc = tokenized_doc.apply(lambda x: [word for word in x if word not in (stop)])


In [8]:
# 저장된 단어들 확인하기 
print(tokenized_doc)

0      [부산, 오수희, 기자, 올해, 부산, 교통사고, 건수, 조금, 사망자, 집계, 부...
1      [여수, 민우, 기자, 뱃길, 마을, 오토바이, 사고, 당한, 남성, 해경이, 긴급...
2      [제네바, 이광철, 특파원, 보험금, 수령, 보험사, 몰카, 허용, 법안, 스위스,...
3      [파주, 숙희, 기자, 경기도, 파주시, 오토바이, 사고, 청년, 경찰, 조사, 중...
4      [전주, 정경, 기자, 오토바이, 사고, 병원, 응급실, 대가, 이틀, 회복, 혼수...
                             ...                        
195    [부산, 김재홍, 기자, 부산, 경찰청, 최근, 교통, 중대사, 예방, 특별, 대책...
196    [홍성, 조성민, 기자, 충남, 자치, 경찰, 위원회, 교통사고, 빅데이터, 분석,...
197    [윤정, 기자, 서울시, 자치, 경찰, 위원회, 서울, 자치, 경찰, 정책, 아이디...
198    [군포, 김인, 기자, 경기, 군포시, 이륜차, 스마트, 관리, 시스템, 바이크, ...
199    [수원, 최종호, 기자, 남부, 경찰청, 관내, 도로, 노출, 암행, 순찰차, 이용...
Length: 200, dtype: object


TF-IDF(Term Frequency - Inverse Document Frequency)는 정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 
여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치

https://wikidocs.net/31698 참고

TF-IDF의 TfidfVectorizer는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용함.
이를 사용하기 위해 다시 토큰화 작업을 역으로 취소하는 역토큰화(Detokenization)작업을 수행

# 사이킷런의 잠재 디리클레 할당(LDA) 실습

https://wikidocs.net/40710

In [9]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []

for i in range(len(sentences)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

sentences['Contents'] = detokenized_doc

print(sentences['Contents'][:5])

0    부산 오수희 기자 올해 부산 교통사고 건수 조금 사망자 집계 부산 경찰청 자료 올해...
1    여수 민우 기자 뱃길 마을 오토바이 사고 당한 남성 해경이 긴급 이송 전남 해양 경...
2    제네바 이광철 특파원 보험금 수령 보험사 몰카 허용 법안 스위스 국민투표 시행 현지...
3    파주 숙희 기자 경기도 파주시 오토바이 사고 청년 경찰 조사 중이 경기 파주 경찰서...
4    전주 정경 기자 오토바이 사고 병원 응급실 대가 이틀 회복 혼수상태 가족 병원 대처...
Name: Contents, dtype: object


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_21040\3324690400.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentences['Contents'] = detokenized_doc


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer 객체 생성
vectorizer = TfidfVectorizer(max_features= 1000)    # 상위 1,000개의 단어를 보존 

# TfidfVectorizer 할 X_train 데이터
X = vectorizer.fit_transform(sentences['Contents'])

from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델훈련
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_model.fit_transform(X)

# vocabulary를 활용하여 각 문장이 갖고 있는 토큰의 count를 기반으로 문장을 vectorization 처리
terms = vectorizer.get_feature_names() 

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

#  토픽별 단어 분포
get_topics(lda_model.components_,terms)



Topic 1: [('이륜차', 1.67), ('단속', 1.61), ('배달', 1.18), ('경찰관', 1.17), ('위반', 1.15), ('경찰', 1.15), ('사고', 1.14), ('운전자', 1.13), ('본부', 1.13), ('전체', 1.12)]
Topic 2: [('오토바이', 1.12), ('심사', 1.06), ('국도', 1.05), ('사고', 1.0), ('바퀴', 1.0), ('병원', 0.99), ('위반', 0.98), ('미착', 0.98), ('과속', 0.98), ('태국', 0.96)]
Topic 3: [('배달', 1.39), ('안전', 1.27), ('사고', 1.13), ('오토바이', 1.12), ('티스', 1.11), ('이륜차', 1.07), ('교통', 1.07), ('주니어', 1.07), ('과실', 1.03), ('차량', 1.02)]
Topic 4: [('이륜차', 1.11), ('제도', 1.08), ('검사', 1.07), ('과실', 1.05), ('스마트', 1.01), ('모든', 1.0), ('폐차', 1.0), ('배달', 0.99), ('툽신바야르', 0.98), ('오후', 0.97)]
Topic 5: [('이륜차', 5.46), ('단속', 4.53), ('사망자', 3.84), ('위반', 3.14), ('사고', 3.02), ('교통사고', 3.01), ('교통', 2.92), ('안전', 2.86), ('경찰', 2.57), ('배달', 2.55)]
Topic 6: [('배달', 1.29), ('종사', 1.18), ('사고', 1.13), ('오토바이', 1.1), ('경찰', 1.0), ('안전모', 1.0), ('경우', 0.99), ('단속', 0.99), ('업체', 0.97), ('가이드라인', 0.96)]
Topic 7: [('경찰', 1.08), ('오토바이', 1.01), ('신씨', 0.99), ('경기도', 0.98), ('설치', 0.97), 

C:\Users\Eddie\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# gensim을 통해서 LDA를 수행하고, 시각화

https://wikidocs.net/30708

In [11]:
from gensim import corpora

dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

In [12]:
import gensim

NUM_TOPICS = 10 # n개의 토픽, k=n

# passes는 알고리즘의 동작 횟수
# 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 설정함
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)   # TOPIC별 단어를 4개만 보여주기, 숫자를 늘리면 더 많은 단어들이 보여짐
for topic in topics:
    print(topic)


(0, '0.017*"장애인" + 0.007*"후보" + 0.007*"사고" + 0.007*"김선미"')
(1, '0.023*"사고" + 0.020*"교통사고" + 0.020*"사망자" + 0.018*"안전"')
(2, '0.031*"사고" + 0.031*"오토바이" + 0.018*"배달" + 0.011*"과실"')
(3, '0.030*"배달" + 0.021*"보험" + 0.013*"가입" + 0.011*"산재"')
(4, '0.019*"티스" + 0.016*"주니어" + 0.010*"샌디에이고" + 0.009*"사고"')
(5, '0.034*"이륜차" + 0.030*"단속" + 0.020*"위반" + 0.019*"사고"')


각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줌
또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 N개의 토픽은 0부터 N-1까지의 번호가 할당되어져 있음

위의 출력 결과에서 (숫자, 확률)은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도를 의미함
예를 들어 0번째 문서의 토픽 비율에서 (0, '0.023*"경제..."'')은 0번 토픽에서 경제가 2.3%의 분포도를 가지는 것을 의미함

In [13]:
# !pip install pyLDAvis

In [14]:

import pyLDAvis.gensim_models
import pyLDAvis.gensim_models as gensimvis


pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)

# https://lovit.github.io/nlp/2018/09/27/pyldavis_lda/
# 시각화는 1- 부터 시작함
pyLDAvis.display(vis)

C:\Users\Eddie\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


λ라는 값은 [0, 1] 사이에서 조절가능함
λ를 1 로 설정하면 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미이며,
λ를 0 에 가깝게 설정할수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미임